In [11]:
import os
import requests
import pandas as pd
import mysql.connector
from datetime import datetime

print("Starting CoinGecko API data fetch...")

# 1. Initialize API parameters
url = "https://api.coingecko.com/api/v3/coins/markets"
pages = 1
per_page = 10

# 2. Create data folder & CSV path
folder = os.path.join(os.getcwd(), "data")
os.makedirs(folder, exist_ok=True)
csv_path = os.path.join(folder, "CoinGecko_API_Data.csv")

# 3. Fetch data from API
total_data = []
for page in range(1, pages + 1):
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": per_page,
        "page": page
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        total_data.extend(response.json())
    else:
        print(f"⚠️ Error occurred when obtaining page {page} (status {response.status_code})")

print("Fetching data from API...")
print(f"✅ Total coins fetched: {len(total_data)}")

# 4. Convert to DataFrame
df = pd.DataFrame(total_data)
df['fetch_date'] = pd.Timestamp.now()

# Select relevant columns
df = df[['id', 'symbol', 'name', 'current_price', 'market_cap', 'market_cap_rank',
         'total_volume', 'high_24h', 'low_24h', 'price_change_24h',
         'price_change_percentage_24h', 'market_cap_change_24h', 'total_supply',
         'ath', 'atl', 'ath_date', 'atl_date', 'fetch_date']]

# Clean up column names for MySQL
df.columns = [col.upper().replace(" ", "_") for col in df.columns]

# Save CSV (append if exists)
df.to_csv(csv_path, mode='a', header=not os.path.exists(csv_path), index=False)
print(f"💾 CSV saved successfully to: {csv_path}")

print("Connecting to MySQL...")

# 5. Connect to MySQL
try:
    conn = mysql.connector.connect(
        host="localhost",      # Use your server name if not local
        user="root",           # Your MySQL username
        password="",           # Your MySQL password (empty if using XAMPP default)
        database="crypto_db"   # Must already exist
    )
    cursor = conn.cursor()

    # 6. Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS crypto_market (
        ID VARCHAR(255),
        SYMBOL VARCHAR(50),
        NAME VARCHAR(255),
        CURRENT_PRICE DOUBLE,
        MARKET_CAP BIGINT,
        MARKET_CAP_RANK INT,
        TOTAL_VOLUME BIGINT,
        HIGH_24H DOUBLE,
        LOW_24H DOUBLE,
        PRICE_CHANGE_24H DOUBLE,
        PRICE_CHANGE_PERCENTAGE_24H DOUBLE,
        MARKET_CAP_CHANGE_24H DOUBLE,
        TOTAL_SUPPLY DOUBLE,
        ATH DOUBLE,
        ATL DOUBLE,
        ATH_DATE DATETIME,
        ATL_DATE DATETIME,
        FETCH_DATE DATETIME
    )
    """
    cursor.execute(create_table_query)
    conn.commit()

    # 7. Insert DataFrame rows into MySQL
    insert_query = """
    INSERT INTO crypto_market (
        ID, SYMBOL, NAME, CURRENT_PRICE, MARKET_CAP, MARKET_CAP_RANK, TOTAL_VOLUME,
        HIGH_24H, LOW_24H, PRICE_CHANGE_24H, PRICE_CHANGE_PERCENTAGE_24H, MARKET_CAP_CHANGE_24H,
        TOTAL_SUPPLY, ATH, ATL, ATH_DATE, ATL_DATE, FETCH_DATE
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    data_tuples = [tuple(None if pd.isna(x) else x for x in row) for row in df.values]
    cursor.executemany(insert_query, data_tuples)
    conn.commit()

    print(f"✅ Successfully inserted {cursor.rowcount} rows into MySQL.")

except mysql.connector.Error as err:
    print("❌ MySQL Error:", err)
except Exception as e:
    print("❌ Unexpected Error:", e)
finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn.is_connected():
        conn.close()

# 8. Preview DataFrame
print(df.head())


Starting CoinGecko API data fetch...


KeyboardInterrupt: 